In [28]:
import pandas as pd
import pickle
import sqlite3
from tqdm.notebook import trange, tqdm
from time import sleep
import dask.dataframe as dd

In [29]:
df_hospitals = pd.read_pickle("hospitals.gz")
con = sqlite3.connect('hospital_pricing.db')
cur = con.cursor()

## Check table

In [30]:
cur.execute(
    "SELECT COUNT(*) FROM prices"
)
prices_length = cur.fetchone()

## Query Data

In [31]:
df_list = pd.read_sql("""Select npi_number, code,
                       case when ip_op like 'OUTPATIENT' then 1
                       when ip_op like 'INPATIENT' then 0
                       ELSE Null END as new_ipop ,
                       price
                       from prices""", con = con, chunksize = 1)       

In [32]:
to_include = df_hospitals['npi_number'].tolist()
result = []
df_prices = pd.DataFrame()

In [33]:
for idx,chunk in tqdm(enumerate(df_list), total = prices_length[0]):
    result.append(chunk.loc[chunk.npi_number.isin(to_include)])
    if (idx%1000000 == 0):
        df_prices = df_prices.append(result)
        result = []
    elif (idx == prices_length):
        df_prices = df_prices.append(result)

  0%|          | 0/100218746 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#For df_procedures
df.loc[df.short_discription.notnull(),'code'].values.tolist()